In [1]:
from DWX_ZeroMQ_Connector_v2_0_1_RC8 import DWX_ZeroMQ_Connector as dwx
import time
import pandas as pd
# from dwx_connector_MVC import DwxModel



# Calculate risk
class risk_management():

    
    def __init__(self, ZMQ_ = None, risk = None, account_info = None, new_trade_df = None, hist_db_key = None):
        
        if risk is None:
            risk = 0.02     # Default value for risk
        self.risk = risk
        if ZMQ_ is None:
            ZMQ_ = dwx()
        self.ZMQ_ = ZMQ_
        self.new_trade_df = new_trade_df
        self.account_info = account_info
        self.hist_db_key = hist_db_key
        self._symbol = hist_db_key[:6]
        self._timeframe = hist_db_key[7:]
        
        self.sec_symbol = None

        #Major Currency pairs, where USD is the secondary currency traded.
        #ADD commodities ie XAG, XPT, XAU
        self.major_curr = ['AUD', 'EUR', 'GBP', 'NZD']

        # Exotic pairs whose values do not conform to typical 5 point values ie. SEK, JPY
        #ADD all necessary pairs to be considered
        self.exotic_curr = ['SEK', 'JPY', 'ZAR', 'XAG', 'XAU', 'XPT']



    # Calculate Margin based on Risk & Stop Loss
    # The formula for calculation is as:
    # Risk amount / Stop Loss Value = Pip Value. This was obtained from the NNFX Trading strategy:
    # https://www.youtube.com/watch?v=bqWLFNpK6eg&t=1161s
    def calc_lot(self):

        try:
            #Subscribe to market data of _symbol
            self.ZMQ_._DWX_MTX_SUBSCRIBE_MARKETDATA_(self._symbol)

            #Request for Current Tracked Prices.
            self.ZMQ_._DWX_MTX_SEND_TRACKPRICES_REQUEST_([self._symbol])

            #Allow time for the values to be loaded onto the Market_DB dict.
            time.sleep(0.05)

            bid_symbol, ask_symbol = self.bid_ask_price(self._symbol)

            #check whether USD is part of the currency pair ('USD' since this is the account currency)
            if 'USD' in self._symbol:
                
                if 'USD' in self._symbol[:3]:
                    pip_value = (bid_symbol + ask_symbol) / 2

                else:
                    pip_value = 1
            
            else:

                #Check whether _symbol contains one of the major pairs (where USD is the secondary currency.)
                #If first part of the symbol is a major currency pair ie. AUD, NZD, EUR, GBP...
                if self._symbol[3:] in self.major_curr:
                    
                    self.sec_symbol = self._symbol[3:] + 'USD'

                    # Collect bid/ask prices from Market_DB
                    bid_sec_symbol, ask_sec_symbol = self.bid_ask_price(self.sec_symbol)

                    pip_value = (bid_sec_symbol + ask_sec_symbol) / 2

                else:
                    self.sec_symbol = 'USD' + self._symbol[3:]

                    # Collect bid/ask prices from Market_DB
                    bid_sec_symbol, ask_sec_symbol = self.bid_ask_price(self.sec_symbol)

                    pip_value = 1 / ((bid_sec_symbol + ask_sec_symbol) / 2)

            #ATR value from dataframe
            atr = self.new_trade_df['atr'].iloc[-1]

            # Calculate risk amount of the accountbalance
            risk = self.account_info['_data']['accountbalance'] * self.risk


            # Calculate pip value for the trade ie. Pip = %Risk Acct. Amount / Risk Stop Loss
            #A00 add functionality for exotic pairs
            if self.exotic_curr not in self._symbol:

                atr_in_pips = atr * 10000

                # Calculate the Pip Value based on the new trade to be taken, ie.
                # Relating the risked amount (%Risk) to the risked pips (factor * ATR_in_pips) 
                calc_pip_value = risk / atr_in_pips

                #To get the lot size, divide the current pip value of the _symbol by the calculated pip value of the new trade
                lot_size = calc_pip_value / pip_value









            stop_loss = self.calc_SL(atr)
            take_profit = self.calc_TP(atr)
                #To Calculate Pip value, sec_symbol / _symbol.
                # ie. in AUDCAD, The pip value is calculated using the rate of AUDCAD / AUDUSD


                    #A00 Include circumstance for JPY & other unique pairs



                    #A00 Include circumstance for JPY & other unique pairs



                # Download the bid/ask price

        except Exception as e:
            print(e)

    # calculate Stop Loss
    #A00 add functionality depending on whether trade is sell or buy, ie using buy or sell value?
    def calc_SL(self, atr):
        # SL calculated as 1.5x of ATR

        #A00 functionality for accepting factor value from user
        factor = 1.5

        return factor * atr

        
        

    # Calculate Take Profit
    def calc_TP(self, atr):
        # TP calculated as 1x of ATR
        factor = 1

        #A00 functionality for accepting factor value from user
        return factor * atr

    # Collect bid & ask prices of specific symbol after downloading current rates
    def bid_ask_price(self, _symbol):

        
        self.ZMQ_._DWX_MTX_SUBSCRIBE_MARKETDATA_(_symbol)
        time.sleep(5)
        
        self.ZMQ_._DWX_MTX_SEND_TRACKPRICES_REQUEST_([_symbol])

        #Pause to allow Market DB to be updated with recent values
        time.sleep(5)

        # Get last item in the dict for the symbol
        bid_symbol, ask_symbol = list(self.ZMQ_._Market_Data_DB.values())[-1]

        self.ZMQ_._DWX_MTX_UNSUBSCRIBE_ALL_MARKETDATA_REQUESTS_()

        self.ZMQ_._DWX_MTX_UNSUBSCRIBE_ALL_MARKETDATA_REQUESTS_()

        self.ZMQ_._DWX_MTX_UNSUBSCRIBE_ALL_MARKETDATA_REQUESTS_()

        return bid_symbol, ask_symbol

    # Create process that monitors specific trade scenarios.

In [2]:
import pandas as pd
import pandas_ta as ta
z = dwx()

[INIT] Ready to send commands to METATRADER (PUSH): 32768
[INIT] Listening for responses from METATRADER (PULL): 32769
[INIT] Listening for market data from METATRADER (SUB): 32770


In [4]:
z._DWX_MTX_SEND_HIST_REQUEST_('AUDCAD')

{'_action': 'HIST', '_symbol': 'AUDCAD_D1', '_data': [{'time': '2020.01.02 00:00', 'open': 0.91021, 'high': 0.91078, 'low': 0.9063, 'close': 0.90756, 'tick_volume': 54483, 'spread': 0, 'real_volume': 0}, {'time': '2020.01.03 00:00', 'open': 0.90729, 'high': 0.90758, 'low': 0.90038, 'close': 0.90314, 'tick_volume': 65527, 'spread': 0, 'real_volume': 0}, {'time': '2020.01.06 00:00', 'open': 0.90133, 'high': 0.90279, 'low': 0.89833, 'close': 0.89965, 'tick_volume': 57217, 'spread': 0, 'real_volume': 0}, {'time': '2020.01.07 00:00', 'open': 0.8995, 'high': 0.89975, 'low': 0.89172, 'close': 0.89304, 'tick_volume': 60130, 'spread': 0, 'real_volume': 0}, {'time': '2020.01.08 00:00', 'open': 0.89269, 'high': 0.8964, 'low': 0.89026, 'close': 0.89509, 'tick_volume': 86444, 'spread': 0, 'real_volume': 0}, {'time': '2020.01.09 00:00', 'open': 0.89499, 'high': 0.89802, 'low': 0.89458, 'close': 0.8953, 'tick_volume': 55214, 'spread': 0, 'real_volume': 0}, {'time': '2020.01.10 00:00', 'open': 0.89527

In [5]:
z._DWX_MTX_GET_ACCOUNT_INFO_()

{'_action': 'GET_ACCOUNT_INFORMATION', '_data': {'currenttime': '2022.01.07 17:53', 'accountname': 'Teddy Njuguna', 'accountnumber': 511588360, 'accountprofit': 39.09, 'accountbalance': 8879.32, 'accountequity': 8918.41, 'accountfreemargin': 5591.69543988, 'accountleverage': 100}}


In [6]:
account_info = z._thread_data_output['_data']

In [7]:
df = pd.DataFrame.from_dict(z._History_DB['AUDCAD_D1'])
df['atr'] = df.ta.atr(length = 14)
hist_db_key = 'AUDCAD_D1'

In [8]:
rm = risk_management(z, _, account_info, df, hist_db_key)

In [9]:
rm.calc_lot()

[KERNEL] Subscribed to AUDCAD BID/ASK updates. See self._Market_Data_DB.
{'_action': 'TRACK_PRICES', '_data': {'symbol_count': 1, 'error_symbols': []}}
[KERNEL] Subscribed to AUDCAD BID/ASK updates. See self._Market_Data_DB.

[AUDCAD] 2022-01-07 15:55:49.290030 (0.907880/0.908150) BID/ASK

[AUDCAD] 2022-01-07 15:55:49.605670 (0.907900/0.908160) BID/ASK

[AUDCAD] 2022-01-07 15:55:50.061738 (0.907900/0.908150) BID/ASK

[AUDCAD] 2022-01-07 15:55:51.564131 (0.907890/0.908150) BID/ASK

[AUDCAD] 2022-01-07 15:55:51.868082 (0.907930/0.908180) BID/ASK
{'_action': 'TRACK_PRICES', '_data': {'symbol_count': 1, 'error_symbols': []}}

[AUDCAD] 2022-01-07 15:55:56.528700 (0.907930/0.908190) BID/ASK
too many values to unpack (expected 2)

[AUDCAD] 2022-01-07 15:55:59.028456 (0.907910/0.908170) BID/ASK

[AUDCAD] 2022-01-07 15:55:59.139122 (0.907900/0.908170) BID/ASK

[AUDCAD] 2022-01-07 15:55:59.917863 (0.907910/0.908170) BID/ASK

[AUDCAD] 2022-01-07 15:56:00.342203 (0.907910/0.908180) BID/ASK

[AUDCA

In [10]:
z._DWX_MTX_UNSUBSCRIBE_ALL_MARKETDATA_REQUESTS_()
z._DWX_MTX_UNSUBSCRIBE_ALL_MARKETDATA_REQUESTS_()


**
[KERNEL] Unsubscribing from AUDCAD
**


**
[KERNEL] Unsubscribing from AUDCAD
**



In [11]:
def return_two():
    return 2,3
a, b = return_two()

In [12]:
a

2

In [13]:
z._Market_Data_DB.keys()

dict_keys(['AUDCAD'])

In [14]:
z._Market_Data_DB.values()

dict_values([{'2022-01-07 15:55:49.290030': (0.90788, 0.90815), '2022-01-07 15:55:49.605670': (0.9079, 0.90816), '2022-01-07 15:55:50.061738': (0.9079, 0.90815), '2022-01-07 15:55:51.564131': (0.90789, 0.90815), '2022-01-07 15:55:51.868082': (0.90793, 0.90818), '2022-01-07 15:55:56.528700': (0.90793, 0.90819), '2022-01-07 15:55:59.028456': (0.90791, 0.90817), '2022-01-07 15:55:59.139122': (0.9079, 0.90817), '2022-01-07 15:55:59.917863': (0.90791, 0.90817), '2022-01-07 15:56:00.342203': (0.90791, 0.90818), '2022-01-07 15:56:00.423152': (0.90793, 0.90819), '2022-01-07 15:56:00.771539': (0.90793, 0.9082), '2022-01-07 15:56:02.330207': (0.90796, 0.90822), '2022-01-07 15:56:02.453223': (0.90795, 0.90822), '2022-01-07 15:56:02.946486': (0.90794, 0.90821), '2022-01-07 15:56:03.447762': (0.90793, 0.9082), '2022-01-07 15:56:05.430787': (0.9079, 0.90813), '2022-01-07 15:56:05.524185': (0.9079, 0.90814), '2022-01-07 15:56:05.926200': (0.90789, 0.90814), '2022-01-07 15:56:07.816819': (0.90787, 0.9

In [15]:
z._DWX_MTX_SUBSCRIBE_MARKETDATA_('EURUSD')

[KERNEL] Subscribed to EURUSD BID/ASK updates. See self._Market_Data_DB.


In [16]:
z._DWX_MTX_SEND_TRACKPRICES_REQUEST_(['EURUSD'])

{'_action': 'TRACK_PRICES', '_data': {'symbol_count': 1, 'error_symbols': []}}

[EURUSD] 2022-01-07 16:01:42.478716 (1.134750/1.134890) BID/ASK

[EURUSD] 2022-01-07 16:01:42.888159 (1.134720/1.134850) BID/ASK

[EURUSD] 2022-01-07 16:01:43.093807 (1.134700/1.134840) BID/ASK

[EURUSD] 2022-01-07 16:01:43.658486 (1.134700/1.134830) BID/ASK

[EURUSD] 2022-01-07 16:01:44.146974 (1.134680/1.134820) BID/ASK

[EURUSD] 2022-01-07 16:01:44.381432 (1.134660/1.134790) BID/ASK

[EURUSD] 2022-01-07 16:01:44.791662 (1.134670/1.134800) BID/ASK

[EURUSD] 2022-01-07 16:01:45.413496 (1.134650/1.134800) BID/ASK

[EURUSD] 2022-01-07 16:01:46.309508 (1.134640/1.134790) BID/ASK

[EURUSD] 2022-01-07 16:01:46.404104 (1.134650/1.134780) BID/ASK

[EURUSD] 2022-01-07 16:01:46.701971 (1.134650/1.134790) BID/ASK

[EURUSD] 2022-01-07 16:01:47.015535 (1.134650/1.134780) BID/ASK

[EURUSD] 2022-01-07 16:01:47.514846 (1.134650/1.134790) BID/ASK

[EURUSD] 2022-01-07 16:01:48.311308 (1.134660/1.134790) BID/ASK

[EURUSD] 2

In [17]:
z._DWX_MTX_UNSUBSCRIBE_ALL_MARKETDATA_REQUESTS_()
z._DWX_MTX_UNSUBSCRIBE_ALL_MARKETDATA_REQUESTS_()


**
[KERNEL] Unsubscribing from AUDCAD
**


**
[KERNEL] Unsubscribing from EURUSD
**


**
[KERNEL] Unsubscribing from AUDCAD
**


**
[KERNEL] Unsubscribing from EURUSD
**



In [18]:
z._Market_Data_DB.keys()

dict_keys(['AUDCAD', 'EURUSD'])